In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from data_pipeline import transformation_pipeline
from sklearn.model_selection import train_test_split
import tensorflow as tf


### Reading Data

In [2]:
data = pd.read_csv('D:/Downloads/final-year-project/data/preprocessed_train_data.csv')

### Creating a pipeline object and cleaning data

In [3]:
pipeline, data_cleaned = transformation_pipeline(
    data, building_id=122, meter=0, primary_use=99)

d:\Downloads\final-year-project\notebooks\data_fetcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['building_id', 'meter', 'primary_use'],
d:\Downloads\final-year-project\notebooks\data_fetcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "timestamp"] = pd.to_datetime(df.loc[:, "timestamp"])
d:\Downloads\final-year-project\notebooks\data_fetcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

### Transforming the data and showing it

In [4]:
transformed_data = pipeline.fit_transform(data_cleaned)

### displaying the meter reading

In [5]:
display(transformed_data[:, 0])

array([0.        , 0.36463482, 0.36902801, ..., 0.2800659 , 0.27594728,
       0.27841845])

### Showing the rest of the data

In [6]:
display(transformed_data[:, 1:])

array([[0.20923913, 0.38175676, 0.65927978, ..., 3.        , 0.        ,
        4.        ],
       [0.20652174, 0.38175676, 0.66897507, ..., 3.        , 0.        ,
        4.        ],
       [0.17663043, 0.36486486, 0.67313019, ..., 3.        , 0.        ,
        4.        ],
       ...,
       [0.29347826, 0.49662162, 0.73268698, ..., 3.        , 1.        ,
        5.        ],
       [0.29347826, 0.51013514, 0.72160665, ..., 3.        , 1.        ,
        5.        ],
       [0.30978261, 0.52027027, 0.71191136, ..., 3.        , 1.        ,
        5.        ]])

### Data Splitting

In [7]:
x_train, x_val, y_train, y_val = train_test_split(transformed_data[:, 1:],
                                                  transformed_data[:, 0],
                                                  test_size=0.2,
                                                  shuffle=False,
                                                  random_state=2021)

### Creating time series data generators

In [8]:
train_gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x_train,
                                                                y_train,
                                                                length=6, sampling_rate=1,
                                                                stride=1, batch_size=32
                                                                )

val_gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x_val,
                                                              y_val,
                                                              length=6, sampling_rate=1,
                                                              stride=1, batch_size=32
                                                              )

### Creating the Model

In [10]:
model = tf.keras.Sequential([tf.keras.layers.LSTM(128, activation='relu',
                                                  return_sequences=False),
                            tf.keras.layers.Dense(1)])

### Training the model

In [11]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.0001))

cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                      patience=15,
                                      restore_best_weights=True)

### Fitting the Model

In [12]:
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=100,
                    callbacks=[cb],
                    shuffle=False)

Epoch 1/100
219/219 [==============================] - 3s 7ms/step - loss: 0.0644 - val_loss: 0.0217
Epoch 2/100
219/219 [==============================] - 1s 6ms/step - loss: 0.0381 - val_loss: 0.0154
Epoch 3/100
219/219 [==============================] - 1s 6ms/step - loss: 0.0233 - val_loss: 0.0143
Epoch 4/100
219/219 [==============================] - 1s 6ms/step - loss: 0.0159 - val_loss: 0.0149
Epoch 5/100
219/219 [==============================] - 1s 6ms/step - loss: 0.0123 - val_loss: 0.0249
Epoch 6/100
219/219 [==============================] - 1s 7ms/step - loss: 0.0106 - val_loss: 0.0362
Epoch 7/100
219/219 [==============================] - 1s 6ms/step - loss: 0.0090 - val_loss: 0.0438
Epoch 8/100
219/219 [==============================] - 1s 5ms/step - loss: 0.0084 - val_loss: 0.0559
Epoch 9/100
219/219 [==============================] - 1s 5ms/step - loss: 0.0084 - val_loss: 0.0647
Epoch 10/100
219/219 [==============================] - 1s 5ms/step - loss: 0.0092 - val_lo

### Model Performance (Before Tuning)

In [13]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Helper function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Helper function to calculate SMAPE
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Assuming you have the generators `train_gen` and `val_gen`
y_true_val = np.concatenate([y for x, y in val_gen], axis=0)
y_pred_val = model.predict(val_gen).flatten()

mse = mean_squared_error(y_true_val, y_pred_val)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true_val, y_pred_val)
mape = mean_absolute_percentage_error(y_true_val, y_pred_val)
smape = symmetric_mean_absolute_percentage_error(y_true_val, y_pred_val)
forecast_bias = np.mean(y_pred_val - y_true_val)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}')
print(f'SMAPE: {smape}')
print(f'Forecast Bias: {forecast_bias}')


55/55 [==============================] - 0s 3ms/step
MSE: 0.006446781877679553
RMSE: 0.08029185436692537
MAE: 0.06263138654604979
MAPE: 18.32984341213807
SMAPE: 20.40293297596518
Forecast Bias: -0.015752323125649056


#### The reason why the tuner not being implemented here is because the tuner do not have learning rate tuning option.

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# Function to create and compile the LSTM model
def build_and_compile_lstm_model(input_shape, lstm_units, dropout, learning_rate, optimizer_choice):
    model = tf.keras.Sequential([
        layers.LSTM(lstm_units, activation='relu', input_shape=input_shape, dropout=dropout),
        layers.Dense(1)
    ])

    # Configure the optimizer based on the selection
    optimizers_dict = {
        'adam': optimizers.Adam(learning_rate=learning_rate),
        'rmsprop': optimizers.RMSprop(learning_rate=learning_rate),
        'sgd': optimizers.SGD(learning_rate=learning_rate)
    }
    optimizer = optimizers_dict[optimizer_choice]

    model.compile(optimizer=optimizer, loss='mse')
    return model

# List of hyperparameters to try
lstm_units_options = [50, 100, 150]
dropout_options = [0.1, 0.2, 0.3]
learning_rate_options = [0.001, 0.01]
optimizer_options = ['adam', 'rmsprop', 'sgd']

# Input shape of the data
input_shape = (6, 12)  # 6 timesteps and 12 features per timestep

# Track the best setting
best_setting = None
best_loss = float('inf')

for units in lstm_units_options:
    for dropout in dropout_options:
        for lr in learning_rate_options:
            for optimizer in optimizer_options:
                # Build and compile the model with current settings
                model = build_and_compile_lstm_model(input_shape, units, dropout, lr, optimizer)

                # Fit the model (assuming train_gen and val_gen are defined)
                history = model.fit(train_gen, validation_data=val_gen, epochs=10, verbose=0)

                # Evaluate the model
                val_loss = model.evaluate(val_gen, verbose=0)

                print(f"Tested: Units={units}, Dropout={dropout}, LR={lr}, Optimizer={optimizer}, Val Loss={val_loss}")

                # Update the best model if current model is better
                if val_loss < best_loss:
                    best_loss = val_loss
                    best_setting = (units, dropout, lr, optimizer)

# Print the best setting after all iterations
print("Best setting found:")
print(f"Units: {best_setting[0]}, Dropout: {best_setting[1]}, Learning Rate: {best_setting[2]}, Optimizer: {best_setting[3]}, Loss: {best_loss}")


Tested: Units=50, Dropout=0.1, LR=0.001, Optimizer=adam, Val Loss=0.014581742696464062
Tested: Units=50, Dropout=0.1, LR=0.001, Optimizer=rmsprop, Val Loss=0.006966582499444485
Tested: Units=50, Dropout=0.1, LR=0.001, Optimizer=sgd, Val Loss=0.030222374945878983
Tested: Units=50, Dropout=0.1, LR=0.01, Optimizer=adam, Val Loss=0.005528555251657963
Tested: Units=50, Dropout=0.1, LR=0.01, Optimizer=rmsprop, Val Loss=0.004463735967874527
Tested: Units=50, Dropout=0.1, LR=0.01, Optimizer=sgd, Val Loss=nan
Tested: Units=50, Dropout=0.2, LR=0.001, Optimizer=adam, Val Loss=0.010578668676316738
Tested: Units=50, Dropout=0.2, LR=0.001, Optimizer=rmsprop, Val Loss=0.007940234616398811
Tested: Units=50, Dropout=0.2, LR=0.001, Optimizer=sgd, Val Loss=0.010313737206161022
Tested: Units=50, Dropout=0.2, LR=0.01, Optimizer=adam, Val Loss=0.0071882507763803005
Tested: Units=50, Dropout=0.2, LR=0.01, Optimizer=rmsprop, Val Loss=0.005468044430017471
Tested: Units=50, Dropout=0.2, LR=0.01, Optimizer=sgd, 

In [9]:
import tensorflow as tf

# Define the model
final_model = tf.keras.Sequential([
    # Adding an LSTM layer with 100 units and applying dropout within the LSTM
    tf.keras.layers.LSTM(100, activation='relu', return_sequences=False, dropout=0.1),
    # Output layer with a single unit (for regression tasks)
    tf.keras.layers.Dense(1)
])

# Compile the model with the specified optimizer and learning rate
final_model.compile(
    loss='mse',  # Mean Squared Error as the loss function
    optimizer=tf.keras.optimizers.Adam(0.01)  # Adam optimizer with a learning rate of 0.01
)

# Callback for early stopping to prevent overfitting
cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

# Assuming 'train_gen' and 'val_gen' are predefined data generators for training and validation
# Fit the model using the training generator and validate using the validation generator
history = final_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[cb],
    shuffle=False  # Important for time series data to maintain sequence integrity
)

# Optional: Print out the history to see how the loss and validation loss progressed
print(history.history)


Epoch 1/100
219/219 [==============================] - 2s 4ms/step - loss: 0.1812 - val_loss: 0.0109
Epoch 2/100
219/219 [==============================] - 1s 4ms/step - loss: 0.0141 - val_loss: 0.0083
Epoch 3/100
219/219 [==============================] - 1s 4ms/step - loss: 0.0113 - val_loss: 0.0092
Epoch 4/100
219/219 [==============================] - 1s 4ms/step - loss: 0.0104 - val_loss: 0.0065
Epoch 5/100
219/219 [==============================] - 1s 4ms/step - loss: 0.0095 - val_loss: 0.0054
Epoch 6/100
219/219 [==============================] - 1s 4ms/step - loss: 0.0087 - val_loss: 0.0064
Epoch 7/100
219/219 [==============================] - 1s 4ms/step - loss: 0.0086 - val_loss: 0.0074
Epoch 8/100
219/219 [==============================] - 1s 4ms/step - loss: 0.0080 - val_loss: 0.0061
Epoch 9/100
219/219 [==============================] - 1s 4ms/step - loss: 0.0078 - val_loss: 0.0060
Epoch 10/100
219/219 [==============================] - 1s 4ms/step - loss: 0.0079 - val_lo

In [10]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Helper function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Helper function to calculate SMAPE
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Assuming you have the generators `train_gen` and `val_gen`
y_true_val = np.concatenate([y for x, y in val_gen], axis=0)
y_pred_val = final_model.predict(val_gen).flatten()

mse = mean_squared_error(y_true_val, y_pred_val)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true_val, y_pred_val)
mape = mean_absolute_percentage_error(y_true_val, y_pred_val)
smape = symmetric_mean_absolute_percentage_error(y_true_val, y_pred_val)
forecast_bias = np.mean(y_pred_val - y_true_val)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}')
print(f'SMAPE: {smape}')
print(f'Forecast Bias: {forecast_bias}')


55/55 [==============================] - 0s 1ms/step
MSE: 0.005391505183610941
RMSE: 0.07342686962965901
MAE: 0.056744420113253605
MAPE: 14.7194370963044
SMAPE: 14.907820619282484
Forecast Bias: -0.015065061250218096


In [11]:

final_model.save('models/LSTM_ADAM')

INFO:tensorflow:Assets written to: models/LSTM_ADAM\assets


INFO:tensorflow:Assets written to: models/LSTM_ADAM\assets
